In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
from copy import deepcopy
import sys
import datasets
import matplotlib.pyplot as plt
import numpy as np
sys.path.append('../experiments/')
import os
import scipy
import pandas as pd
import joblib
import numpy as np
import transformers
import sys
from os.path import join
import datasets
from dict_hash import sha256
import numpy as np
from torch.autograd import grad
import torch
from tqdm import tqdm
from torch.autograd.functional import jacobian
from torch.func import jacfwd
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.utils.data import DataLoader, Dataset
import torchhd
from data import BabyLMDataset, BABYLM_ROOT_DIR
from lm import HDLM
from collections import defaultdict

# Basic sanity check

In [ ]:
checkpoint = 'gpt2'
tok = AutoTokenizer.from_pretrained(checkpoint)
start = 'Roses are red, violets are'
end = ' blue'
lm = HDLM(checkpoint, device='cpu', emb_size=10000, learning_rate=0.1)

print('tokens as strings', lm.tokenizer.tokenize(start))
input_ids = lm.tokenizer.encode(start, return_tensors='pt').to(lm.device)
print('tokens as ids', input_ids)

lm.get_embs(input_ids).shape

next_emb = lm.next_emb_from_token_ids(input_ids)
print('next_emb', next_emb.shape)

next_token_id = lm.emb_to_token_id(next_emb)
print('next_token original', next_token_id, lm.tokenizer.decode(next_token_id))

# update vocab emb
next_token_correct_id = lm.tokenizer(end)['input_ids']
lm.update_vocab_emb(next_emb, next_token_correct_id)


next_token_id = lm.emb_to_token_id(next_emb)
print('next_token', next_token_id, lm.tokenizer.decode(next_token_id))
assert lm.tokenizer.decode(next_token_id) == end

# Training loop

In [ ]:
checkpoint = 'gpt2'
device = 'cuda'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
context_length = 3
learning_rate = 0.5  # 0.01

# initialize
lm = HDLM(checkpoint, device=device, emb_size=50000,
          learning_rate=learning_rate, context_length=context_length)
dset = BabyLMDataset(join(BABYLM_ROOT_DIR, 'babylm_dev',
                     'full.joblib'), context_length)
dset_test = BabyLMDataset(
    join(BABYLM_ROOT_DIR, 'babylm_test', 'full.joblib'), context_length)

In [ ]:
batch_idx = 0
d = defaultdict(list)
test_kwargs = dict(dset=dset_test, n_examples=200, train=False, seed=13)
train_kwargs = dict(dset=dset, n_examples=200, train=True)
for batch_num in tqdm(range(100)):

    # test
    perplexity_test = lm.calc_perplexity(**test_kwargs)
    d['perplexity_test'].append(perplexity_test)
    d['batch_num'].append(batch_num)

    # train
    perplexity_train = lm.calc_perplexity(**train_kwargs, seed=0)

    print('perplexity_test', perplexity_test)
    print('perplexity_train', perplexity_train)

In [ ]:
# run an example
toks, next = dset[0]
print(repr(lm.tokenizer.decode(toks)), '->', repr(lm.tokenizer.decode(next)))
emb_pred = lm.next_emb_from_token_ids(toks)
tok_pred = lm.tokenizer.decode(lm.emb_to_token_id(emb_pred))
print('pred', repr(tok_pred))